In [1]:
!python --version


Python 3.10.18


In [2]:
import pandas as pd
import numpy as np
import joblib
import tarfile
import sagemaker
from sklearn.ensemble import RandomForestClassifier
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import get_execution_role

# Initialize SageMaker session
session = sagemaker.Session()
bucket = session.default_bucket()
role = get_execution_role()


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
data = []
for _ in range(1000):
    temp = round(np.random.uniform(20, 40), 2)
    humidity = round(np.random.uniform(50, 90), 2)
    soil = round(np.random.uniform(5, 40), 2)
    irrigation = 1 if soil < 15 and temp > 30 else 0
    data.append([temp, humidity, soil, irrigation])

df = pd.DataFrame(data, columns=["temperature", "humidity", "soil_moisture", "irrigation_needed"])
df.to_csv("sensor_training_data.csv", index=False)
df.head()


,temperature,humidity,soil_moisture,irrigation_needed
0,33.22,61.33,17.29,0
1,22.56,77.01,21.16,0
2,32.03,73.79,17.18,0
3,27.72,55.14,34.43,0
4,21.16,77.29,34.11,0


In [4]:
df = pd.read_csv("sensor_training_data.csv")
X = df[["temperature", "humidity", "soil_moisture"]]
y = df["irrigation_needed"]

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

joblib.dump(model, "irrigation_model.joblib")
print(" Model trained and saved")


 Model trained and saved


In [5]:
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("irrigation_model.joblib", arcname="irrigation_model.joblib")
print(" Model packaged as model.tar.gz")


 Model packaged as model.tar.gz


In [6]:
s3_path = session.upload_data("model.tar.gz", bucket=bucket, key_prefix="models")
print("Model uploaded to:", s3_path)


Model uploaded to: s3://sagemaker-us-east-1-196936075057/models/model.tar.gz


In [ ]:
model = SKLearnModel(
    model_data=s3_path,
    role=role,
    entry_point="predict.py",
    framework_version="1.0-1",  # ✅ Supports Python 3.8+
    py_version="py3"
)

predictor = model.deploy(
    instance_type="ml.m5.large",
    initial_instance_count=1
)

print("Model deployed successfully!")


---